In [21]:
import sys
sys.path.append('../')

In [22]:
from tft.data import DataLoader

In [23]:
dt = DataLoader(data_dir='../../data/').load_all()

In [24]:
def winrate(x):
    return x.where(x == True).count() / len(x)

def true_count(x):
    return x.where(x == True).count()

def winrate_for(df, group: str, cutoff = 1):
    df['winrate'] = df['placement'] <= cutoff
    df['win'] = df['placement'] <= cutoff
    df['loss'] = df['placement'] >= cutoff
    wr = df.groupby(group).agg({'winrate': winrate, 'win': true_count, 'loss': true_count})
    return wr.sort_values(by=['winrate'], ascending=False)
    

In [25]:
winrate_for(dt.units, 'character_id')

,winrate,win,loss
character_id,,,
TFT7_AoShin,0.235744,616,2613
TFT7_Pyke,0.228584,539,2358
TFT7_Zoe,0.216189,1477,6832
TFT7_Bard,0.215106,1891,8791
TFT7_Yasuo,0.203851,1588,7790
TFT7_Soraka,0.196980,1148,5828
TFT7_Aatrox,0.190533,161,845
TFT7_Kayn,0.179762,302,1680
TFT7_Shyvana,0.170016,423,2488


In [26]:
winrate_for(dt.items, 'item')

,winrate,win,loss
item,,,
TFT_Item_EmptyBag,0.726141,350,482
TFT7_Item_ShimmerscaleCrownOfChampions,0.390023,172,441
TFT5_Item_FrozenHeartRadiant,0.363636,4,11
TFT5_Item_LocketOfTheIronSolariRadiant,0.333333,8,24
TFT7_Item_ShimmerscaleEmblemItem,0.311382,435,1397
...,...,...,...
TFT7_Item_ShimmerscaleHeartOfGold_HR,0.033333,1,30
TFT7_Item_ShimmerscaleDeterminedInvestor,0.015075,3,199
TFT5_Item_RabadonsDeathcapRadiant,0.000000,0,13


In [27]:
winrate_for(dt.augments, 'augment')

,winrate,win,loss
augment,,,
TFT7_Augment_MageConferenceHR,0.500000,1,2
TFT7_Augment_LivingForgeHR,0.400000,6,15
TFT7_Augment_ShimmerscaleEmblem,0.328358,110,335
TFT7_Augment_ShimmerscaleTrait2,0.326923,34,104
TFT7_Augment_ShimmerscaleTrait,0.298942,113,378
...,...,...,...
TFT6_Augment_TheGoldenEggHR,0.000000,0,3
TFT7_Augment_RevelPartyFavorsHR,0.000000,0,5
TFT7_Augment_Preparation3HR,0.000000,0,4


In [28]:
star3_df = dt.units.loc[:][dt.units['tier'] == 3]
winrate_for(star3_df, 'character_id')

,winrate,win,loss
character_id,,,
TFT7_AurelionSol,1.000000,6,6
TFT7_Soraka,0.939394,31,33
TFT7_Zoe,0.933333,28,30
TFT7_Yasuo,0.903226,28,31
TFT7_AoShin,0.866667,13,15
TFT7_Pyke,0.829268,34,41
TFT7_Shyvana,0.800000,4,5
TFT7_Bard,0.761905,16,21
TFT7_Xayah,0.654762,55,84
